In [19]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook, tqdm_pandas
import random

import re

from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import keras
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score

from tensorflow.python.client import device_lib
import ast
from ast import literal_eval

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

In [20]:
def get_clf_eval(y_test, pred):
    score = []
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
    specificity = tn/(tn + fp)
    roc_auc = roc_auc_score(y_test, pred)

    score = [accuracy, precision, recall, f1, specificity, roc_auc, tn, tp, fn, fp]
    return score
def get_mean(score):
    for i in range(5):
        accuracy = score[i][0]
        precision = score[i][1]
        recall = score[i][2]
        f1 = score[i][3]
        specificity = score[i][4]
        roc_auc = score[i][5]
        tn = score[i][6]
        tp = score[i][7]
        fn = score[i][8]
        fp = score[i][9]

    print("accuracy: "+str(np.mean(accuracy))+", precision: "+str(np.mean(precision))+", recall: "+str(np.mean(recall))+", f1: "+str(np.mean(f1))+", specificity"+str(np.mean(specificity))+", roc_auc: "+str(np.mean(roc_auc))+", tn: "+str(np.mean(tn)) + "tp: "+str(np.mean(tp)) + "fn: "+str(np.mean(fn)) + "fp: "+str(np.mean(fp)))
    return ("accuracy: "+str(np.mean(accuracy))+", precision: "+str(np.mean(precision))+", recall: "+str(np.mean(recall))+", f1: "+str(np.mean(f1))+", specificity"+str(np.mean(specificity))+", roc_auc: "+str(np.mean(roc_auc))+
           ", tn: "+str(np.mean(tn)) + ", tp: "+str(np.mean(tp)) + ", fn: "+str(np.mean(fn)) + ", fp: "+str(np.mean(fp)))


## Experience 1 -> all data

In [6]:
df = pd.read_csv('use_data/final_use.csv', encoding = 'utf-8-sig', index_col = 0)

In [16]:
X = df[['review_rating', 'Affect', 'Cognition',
       'Social', 'Perception', 'motion', 'Psychological Distancing','depth', 'style', 'review_extremity', 'photo',
       'text_structure', 'reputation', 'experience', 'location' ]]

y = df[['fake']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 215, stratify=y)

In [17]:
X_train.reset_index(inplace = True, drop=True)
X_test.reset_index(inplace = True, drop=True)
y_train.reset_index(inplace = True, drop=True)
y_test.reset_index(inplace = True, drop=True)

In [21]:
xg_clssifier = xgb.XGBClassifier(n_estimators=100)
xg_clssifier.fit(X_train, y_train)
pred_xgb = xg_clssifier.predict(X_test)
get_clf_eval(y_test, pred_xgb)

[0.9197152003532398,
 0.6632855022266205,
 0.311998138019318,
 0.42437673130193904,
 0.9834018317743332,
 0.6476999848968256,
 80636,
 2681,
 5912,
 1361]

In [22]:
svm_clasiifier = SVC(max_iter=100)

svm_clasiifier.fit(X_train, y_train)
pred_svm = svm_clasiifier.predict(X_test)
get_clf_eval(y_test, pred_svm)

[0.25824042388784635,
 0.1096724346934154,
 0.9581054346561154,
 0.19681575864789272,
 0.18489700842713758,
 0.5715012215416265,
 15161,
 8233,
 360,
 66836]

In [23]:
rf_classifier = RandomForestClassifier(n_estimators = 100)
rf_classifier.fit(X_train, y_train)

pred_rf = rf_classifier.predict(X_test)
get_clf_eval(y_test, pred_rf)

[0.9187106744673805,
 0.6613284326594907,
 0.29314558361457,
 0.40622480245121756,
 0.9842677171116017,
 0.6387066503630858,
 80707,
 2519,
 6074,
 1290]

In [24]:
LR = LogisticRegression()
LR.fit(X_train, y_train)
pred_lr = LR.predict(X_test)
get_clf_eval(y_test, pred_lr)

[0.9074401148029584,
 0.6947565543071161,
 0.04317467706272547,
 0.08129724991782622,
 0.9980121223947218,
 0.5205933997287236,
 81834,
 371,
 8222,
 163]

In [25]:
from sklearn.neighbors import KNeighborsClassifier

KNN = KNeighborsClassifier()
KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
get_clf_eval(y_test, pred_KNN)

[0.904150568495419,
 0.488905325443787,
 0.23076923076923078,
 0.3135425725353783,
 0.9747185872653877,
 0.6027439090173092,
 79924,
 1983,
 6610,
 2073]